In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM articles ORDER BY datePublished", sql)

In [ ]:
df["full_text"] = df["title"] + "\n" + df["header"] + "\n" + df["text"]

In [ ]:
!pip install textacy
!python -m spacy download de_core_news_lg

In [ ]:
import spacy

nlp = spacy.load("de_core_news_lg", disable=("parser",))

In [ ]:
import textacy

corpus = textacy.Corpus(nlp, data = list(df["full_text"].values))

In [ ]:
corpus.n_docs, corpus.n_sents, corpus.n_tokens

In [ ]:
df["tokens"] = [len(d) for d in corpus]

In [ ]:
df["characters"] = df["full_text"].str.len()

In [ ]:
df.describe()

In [ ]:
!pip install matplotlib

In [ ]:
df[["tokens"]].plot.hist()

In [ ]:
df[["tokens"]].plot.hist(bins=40)

In [ ]:
df[["characters"]].plot.hist(bins=40, logy=True)